# Binary & Semantic segmentation



Today:
- segmentation metrics
- simple unet
- catalyst config & segmentation

## Metrics
[simple explanation](https://towardsdatascience.com/metrics-to-evaluate-your-semantic-segmentation-model-6bcb99639aa2)
- Pixelwise accuracy (bad for imbalanced dataset as classification accuracy)
- IoU = Area of intersection(A, B) / Area of union(A, B) (for binary masks of given class C)
- Dice = 2 * Area of intersection(A, B) / (Area(A)+Area(B)) (for binary masks of given class C)

[iou vs dice](https://stats.stackexchange.com/questions/273537/f1-dice-score-vs-iou/276144#276144)
- IoU and Dice are positively correlated (which means for same A and B, dice(model1) < dice(model2) <-> IoU(model1) < IoU(model2))
- for multiple instances: "Suppose for example that the vast majority of the inferences are moderately better with classifier A than B, but some of them of them are significantly worse using classifier A. It may be the case then that the F metric favors classifier A while the IoU metric favors classifier B."

IoU returns measure **closer to worst-performing** (similar to L2)

Dice returns **closer to mean performance** (similar to L1)

Alternative name for dice is f-measure (f1-score)

Alternative name for IoU is Jaccard Index

In [ ]:
import torch

outputs = torch.tensor([
    [
        [0, 0, 0, 1],
        [1, 0, 0, 1],
        [1, 1, 0, 1]
    ]
])[:, None]
targets = torch.tensor([
    [
        [0, 0, 0, 1],
        [1, 1, 0, 1],
        [1, 1, 0, 1]
    ]
])[:, None]
outputs.shape, targets.shape

In [ ]:
def approx_eq(a, b, eps=1e-4):
    return ((a-b).abs().sum() < eps).item()

# Metrics
Implement IoU, Dice and pixelwise accuracy

In [ ]:
def accuracy(
        outputs,
        targets
):
    """
    Computes the accuracy for 2 tensors of same shape
    
    accuracy = n_correct / n_total
    """
    # TODO(students)
    accuracy = ?
    return accuracy

accuracy_metric = accuracy(outputs, targets)
assert approx_eq(accuracy_metric, 0.9167)
accuracy_metric

In [ ]:
from catalyst.utils import get_activation_fn

def dice(
    outputs: torch.Tensor,
    targets: torch.Tensor,
    eps: float = 1e-7,
    threshold: float = None,
    activation: str = "none"
):
    """
    Computes the dice metric
    
    dice = 2 * intersection / (union + intersection)

    Args:
        outputs (tensor):  A tensor of predicted elements
        targets (tensor): A tensor of elements that are to be predicted
        eps (float): epsilon
        threshold (float): threshold for outputs binarization
        activation (str): An torch.nn activation applied to the outputs.
            Must be one of ["none", "Sigmoid", "Softmax2d"]

    Returns:
        double:  Dice score
    """
    activation_fn = get_activation_fn(activation)
    outputs = activation_fn(outputs)

    if threshold is not None:
        outputs = (outputs > threshold)
        
    outputs = outputs.float()
    targets = targets.float()
    # TODO(students)
    dice = ?

    return dice

dice_metric = dice(outputs, targets)
assert approx_eq(dice_metric, 0.9231)
dice_metric

In [ ]:
def iou(
    outputs: torch.Tensor,
    targets: torch.Tensor,
    eps: float = 1e-7,
    threshold: float = None,
    activation: str = "none"
):
    """
    iou = intersection / union
    
    Args:
        outputs (torch.Tensor): A list of predicted elements
        targets (torch.Tensor):  A list of elements that are to be predicted
        eps (float): epsilon to avoid zero division
        threshold (float): threshold for outputs binarization
        activation (str): An torch.nn activation applied to the outputs.
            Must be one of ["none", "Sigmoid", "Softmax2d"]

    Returns:
        float: IoU (Jaccard) score
    """
    activation_fn = get_activation_fn(activation)
    outputs = activation_fn(outputs)

    if threshold is not None:
        outputs = (outputs > threshold)
        
    outputs = outputs.float()
    targets = targets.float()

    # TODO(students)
    iou = ?

    return iou

iou_metric = iou(outputs, targets)
assert approx_eq(iou_metric, 0.8571)
iou_metric

## U-net

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

class ConvBlock(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.conv(x)


class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            ConvBlock(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(in_channels // 2, in_channels // 2, kernel_size=2, stride=2)

        self.conv = ConvBlock(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        # p2d = (1, 1, 2, 2) pads last dim by (1, 1) and 2nd to last by (2, 2)
        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        # if you have padding issues, see
        # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
        # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=True):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = ConvBlock(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        self.down4 = Down(512, 512)
        self.up1 = Up(1024, 256, bilinear=True)
        self.up2 = Up(512, 128, bilinear=True)
        self.up3 = Up(256, 64, bilinear=True)
        self.up4 = Up(128, 64, bilinear=True)
        self.outc = OutConv(64, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits
    
UNet(3, 10)

## Catalyst again

You will be working with [this](http://artelab.dista.uninsubria.it/downloads/datasets/barcode/medium_barcode_1d/medium_barcode_1d.html) dataset. You can also try PascalVoc2012.

1. unzip ./data/Artelab.zip
2. sh ./scripts/prepare_data.sh
3. catalyst-dl run -C /path/to/config --logdir /path/to/logdir --baselogdir /path/to/baselogdir
4. cat runs.sh | catalyst-parallel-run <n_gpus>

[catalyst explained (video)](https://youtu.be/SGawkIjBoGE?t=561)

[catalyst explained (slides)](https://docs.google.com/presentation/d/10dJqTGEPxk_gYKCZZdFqHHhuPUwwKNYdRhvuNphpy5E/edit#slide=id.g5eb386574d_0_155)